In [ ]:
#pip install opencv-python

In [1]:
import cv2
import numpy as np
cover_path='.\Test-Videos-Image\q2-cover_vid.mp4'
secret_path='.\Test-Videos-Image\q2-secret_vid.mp4'
def VideoToTensor(path):
    cap = cv2.VideoCapture(path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break  
    
        frame_array = np.array(frame)
        frames.append(frame_array)

    cap.release()
    tensor_video= np.array(frames) 
    return tensor_video


In [2]:
cover_video=VideoToTensor(cover_path)
secret_video=VideoToTensor(secret_path)

In [3]:
C = np.transpose(cover_video, (1, 2, 0, 3))
S = np.transpose(secret_video, (1, 2, 0, 3))

In [4]:
C.shape

(144, 144, 20, 3)

In [5]:
S.shape

(144, 144, 20, 3)

In [6]:
import numpy as np
from scipy.linalg import qr
def QRdecomposition(A):
    m,n,p,q=A.shape
    A_prm = A.copy()
    for i in [2,3]:
        A_prm=np.fft.fft(A_prm,axis=i)
    k=p*q
    
    Q_prm=np.zeros((m,n,p,q),dtype=complex)
    R_prm=np.zeros((m,n,p,q),dtype=complex)
    for i in range(p):
        for j in range(q):
            Q_t,R_t=qr(A_prm[:,:,i,j])
            Q_prm[:,:,i,j]=Q_t
            R_prm[:,:,i,j]=R_t
            
    Q=Q_prm.copy()
    R=R_prm.copy()
    for i in [3,2]:
        Q=np.fft.ifft(Q,axis=i)
        R=np.fft.ifft(R,axis=i)
    return Q.real,R.real
        
        

In [7]:
Qc,Rc=QRdecomposition(C)

In [8]:
Qs,Rs=QRdecomposition(S)

In [9]:
def t4Product(A,B):
    m,n,p,q=A.shape
    m1,n1,p1,q1=B.shape
    AFFT=A.copy()
    BFFT=B.copy()
    for i in [2,3]:
        AFFT=np.fft.fft(AFFT,axis=i)
        BFFT=np.fft.fft(BFFT,axis=i)

    CFFT=np.zeros((m,n1,p,q),dtype=complex)
    for i in range(p):
        for j in range(q):
            CFFT[:,:,i,j]=np.dot(AFFT[:,:,i,j],BFFT[:,:,i,j])
    C=CFFT.copy()        
    for i in [3,2]:
        C=np.fft.ifft(C,axis=i)
        
    
    return C.real
    

In [10]:
Rt=Rc+(0.1*Rs)


In [11]:
T=t4Product(Qc,Rt)

In [12]:
T.shape

(144, 144, 20, 3)

In [13]:
Transfer_video = np.transpose(T, (2, 0, 1, 3))    

In [14]:
Transfer_video.shape

(20, 144, 144, 3)

In [15]:
Qt,Rtt=QRdecomposition(T)

In [28]:
def save_video(tensor, filename="output.mp4", fps=30):
    
    T, H, W, C = tensor.shape
    assert C in [1, 3], "Only grayscale (C=1) or RGB (C=3) supported."

    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(filename, fourcc, fps, (W, H))

    for t in range(T):
        frame = tensor[t]  
        frame = (frame).astype(np.uint8) 
        if C == 3:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) 
        out.write(frame)

    out.release()
    print(f"Video saved as {filename}")


save_video(Transfer_video, "Output\Q2\CoverVideoTranfered-TProduct-Output.mp4", fps=5)

Video saved as Output\Q2\CoverVideoTranfered-TProduct-Output.mp4


In [23]:
Ro=(Rtt-Rc)/0.1

In [24]:
O=t4Product(Qs,Ro)

In [25]:
O.shape

(144, 144, 20, 3)

In [26]:
Org_video = np.transpose(O, (2, 0, 1, 3)) 
Org_video.shape

(20, 144, 144, 3)

In [29]:
save_video(Org_video, "Output\Q2\OriginalVideoTranfered-TProduct-Output.mp4", fps=5)

Video saved as Output\Q2\OriginalVideoTranfered-TProduct-Output.mp4
